This notebook constructs state and action objects from the log tables, and exports them as transition tables and lists in [pickle](https://docs.python.org/3/library/pickle.html) format.

## Initialization

In [1]:
import pickle

import pathlib as pl
import pandas as pd
import networkx as nx

from justhink_world import create_all_worlds
from justhink_world.domain.state import NetworkState, EnvState
from justhink_world.agent import Agent
from justhink_world.domain.action import *

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### Define paths.

In [3]:
resources_dir = pl.Path('resources')
networks_dir = resources_dir.joinpath('networks')
logs_dir = pl.Path('../processed_data/logs')

# Outputs.
transitions_pickle_file = pl.Path(
    '../processed_data/justhink21_transitions.pickle')
transition_lists_pickle_file = pl.Path(
    '../processed_data/justhink21_transition_lists.pickle')

### Load activity instances.
Activities are represented in 'justhink_world' as worlds.

In [4]:
worlds = create_all_worlds()

# Print information on the activities.
# The name of the activity.
# name = 'pretest-1'
name = 'collaboration-1'

# Background network connections and costs for that activity.
print(worlds[name].env.state.network.graph.edges(data=True))

[(3, 1, {'cost': 3}), (3, 0, {'cost': 3}), (3, 2, {'cost': 3}), (3, 5, {'cost': 4}), (3, 7, {'cost': 5}), (1, 9, {'cost': 3}), (1, 2, {'cost': 2}), (1, 5, {'cost': 4}), (1, 4, {'cost': 3}), (0, 2, {'cost': 5}), (0, 7, {'cost': 4}), (2, 9, {'cost': 4}), (2, 8, {'cost': 3}), (5, 4, {'cost': 2}), (5, 7, {'cost': 3}), (5, 6, {'cost': 4}), (4, 9, {'cost': 3}), (4, 6, {'cost': 2}), (7, 6, {'cost': 2}), (9, 8, {'cost': 2})]


In [5]:
# Construct an activity name map from the names in the logs
# the latest names in the justhink_world representation package.
world_renamer = {name: name for name in worlds}
world_renamer['indiv-illustrate'] = 'tutorial'
world_renamer['collab-activity'] = 'collaboration-1'
world_renamer['collab-activity-2'] = 'collaboration-2'
world_renamer['debriefing'] = 'bye'

world_renamer

{'intro': 'intro',
 'tutorial': 'tutorial',
 'pretest-1': 'pretest-1',
 'pretest-2': 'pretest-2',
 'pretest-3': 'pretest-3',
 'pretest-4': 'pretest-4',
 'pretest-5': 'pretest-5',
 'posttest-1': 'posttest-1',
 'posttest-2': 'posttest-2',
 'posttest-3': 'posttest-3',
 'posttest-4': 'posttest-4',
 'posttest-5': 'posttest-5',
 'collaboration-1': 'collaboration-1',
 'collaboration-2': 'collaboration-2',
 'bye': 'bye',
 'robot-individual-1': 'robot-individual-1',
 'indiv-illustrate': 'tutorial',
 'collab-activity': 'collaboration-1',
 'collab-activity-2': 'collaboration-2',
 'debriefing': 'bye'}

### Read the state transition log files.

In [6]:
# Discover the sate transition files.
file_list = logs_dir.glob(
    'justhink21_log_*/justhink_app-state_transition.csv')

# Make a map from participant to log file path.
files = {}
for f in file_list:
    i = int(str(f.parent).split('_')[-1])
    files[i] = f

# Read the log files.
logs = {}
for participant in files:
    df = pd.read_csv(files[participant])
    logs[participant] = df

## Processing

### Construct a state object from a log table row.

In [7]:
def construct_state(row, graph, state_name='state', verbose=False):
    """Construct a state object from a log table row.

    Note: The argument state_name='state' for the current state, 
    state_name='next_state' for the next state in
    the 3-tuple logs of <state, action, next_state>
    """
    # Reconstruct the list of selected edges from string.
    edge_list_text = str(row['{}.edges'.format(state_name)])
    edge_list = edge_list_text.strip('[').strip(']').split(',')
    edges = list()
    for edge_text in edge_list:
        # if the edge list is empty, no edges are selected.
        if edge_list[0] == '':
            break
        l = edge_text.split()
        edge = (int(l[1]), int(l[3]))
        edges.append(edge)
    # Represent the selected edges as a subgraph.
    subgraph = nx.Graph()
    for u, v in edges:
        subgraph.add_edge(u, v)

    # Construct the suggested edge.
    suggested_edge = (int(row['{}.suggested.u'.format(state_name)]),
                      int(row['{}.suggested.v'.format(state_name)]))
    if suggested_edge == (-1, -1):
        suggested_edge = None

    # Construct the terminal/not and submitting/not information.
    is_terminal = bool(row['{}.terminal'.format(state_name)])
    is_submitting = bool(row['{}.submit_suggested'.format(state_name)])

    # Construct the set of active agents (indicating the turn at that state).
    if row['turn_agent'] == 'human':
        agents = frozenset({Agent.HUMAN})
    elif row['turn_agent'] == 'robot':
        agents = frozenset({Agent.ROBOT})
    else:
        agents = frozenset()

    # Create the network state object.
    network = NetworkState(graph, subgraph, suggested_edge)

    # Create the environment state object.
    state = EnvState(
        network, agents=agents, is_submitting=is_submitting, is_terminal=is_terminal)

    if verbose:
        print(state)

    return state


# Example / try out:
world = worlds['pretest-1']
df = logs[1].copy()
row_index = 10
construct_state(df.iloc[row_index],
                world.env.state.network.graph, verbose=True)

Situation(Network(|E'|=6 in G(|V|=7, |E|=12)) where |V'|=6 with cost=19, Human)


Situation(Network(|E'|=6 in G(|V|=7, |E|=12)) where |V'|=6 with cost=19, Human)

### Construct an action object from a log table row.

In [8]:
def construct_action(row, graph, prev_action=None, verbose=False):
    """Construct an action object from a log table row."""
    action_type = row['action.type']
    if row['action.agent_name'] == 'human':
        agent = Agent.HUMAN
    elif row['action.agent_name'] == 'robot':
        agent = Agent.ROBOT
    else:
        raise NotImplementedError

    if action_type == 0:
        return SuggestPickAction((row['action.edge.u'], row['action.edge.v']), agent)
    elif action_type == 1:
        return PickAction((row['action.edge.u'], row['action.edge.v']), agent)
    elif action_type == 2:
        return UnpickAction((row['action.edge.u'], row['action.edge.v']), agent)
    #if action_type == 3:
    #    return SubmitAction(agent)
    elif action_type == 3 or action_type == 4:
        # Remap attempt-submit to submit (for the tests).
        if 'test' in row['header.frame_id']:
            if  isinstance(prev_action, AttemptSubmitAction):
                return SubmitAction(agent)
            else:
                return AttemptSubmitAction(agent)
        elif action_type == 3 and agent == Agent.ROBOT:
            return AttemptSubmitAction(agent)
        else:
            return SubmitAction(agent)
    elif action_type == 5:
        return ContinueAction(agent)
    elif action_type == 6:
        return AgreeAction(agent)
    elif action_type == 7:
        return DisagreeAction(agent)
    elif action_type == 8:
        return ClearAction()
    return None


# Example / try out:
world = worlds['pretest-1']
df = logs[1].copy()
row_index = 10
construct_action(
    df.iloc[row_index], world.env.state.network.graph, verbose=True)

Action(pick(5,6),Human)

### Identify a submission action and the associated submitted state.

In [9]:
def identify_submission_state(row):
    """Identify which problem state has been submitted"""
    # check if the last state's action type is submit type 3 or 4
    problem_name = row['header.frame_id']

    # Renamed activities.
    problem_name = world_renamer[problem_name]

    action_type = row['action.type']

    if 'test' in problem_name:
        return action_type == 4

    # For collaborative activities check for 3 only.
    elif 'collab' in problem_name:
        return action_type == 3

    else:
        return False

### Construct all states and actions for a log table.

Iterate through the rows.
Also append world_state, action, is_submission, cost, and is_mst columns to the table.

In [10]:
def construct_state_from_table(df, state_name='state', verbose=False):
    """Construct all states and action for a log table.

    Note: The argument state_name='state' for the current state, 
    state_name='next_state' for the next state in
    the 3-tuple logs of <state, action, next_state>
    """
    state_list = []
    attempt_numbers = dict()
    for i, row in df.iterrows():
        # Rename the activity to match with the updated representation package.
        activity = row['header.frame_id']
        activity = world_renamer[activity]
        world = worlds[activity]

        if activity not in attempt_numbers:
            attempt_numbers[activity] = 1

        state = construct_state(
            row, world.env.state.network.graph, state_name=state_name)

        is_submission = identify_submission_state(row)

        # Update the attempt number.
        state.attempt_no = attempt_numbers[activity]
        if is_submission and 'collab' in activity:
            attempt_numbers[activity] = attempt_numbers[activity] + 1

        # Update the max attempts.
        if 'collab' in activity:
            state.max_attempts = 4

        state_list.append(state)

    df[state_name] = state_list

    return df

# Example / try out:
# name = 'collab-activity'
name = 'pretest-1'
df = logs[1].copy()
df = construct_state_from_table(df, state_name='next_state')
df = construct_state_from_table(df, verbose=True)
df.head()

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,state.edges,state.suggested.u,state.suggested.v,state.terminal,state.submit_suggested,action.agent_name,action.type,action.edge.u,action.edge.v,next_state.edges,next_state.suggested.u,next_state.suggested.v,next_state.terminal,next_state.submit_suggested,action_no,step_no,turn_agent,next_state,state
0,1.623049e+09,1,1623048864,591364622,indiv-illustrate,[],-1,-1,False,False,human,1,1,9,[u: 1\nv: 9],-1,-1,False,False,0,1,human,"Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w...","Situation(Network(|E'|=0 in G(|V|=2, |E|=1)) w..."
1,1.623049e+09,2,1623048901,478822946,indiv-illustrate,[u: 1\nv: 9],-1,-1,False,False,human,8,-1,-1,[],-1,-1,False,False,1,2,human,"Situation(Network(|E'|=0 in G(|V|=2, |E|=1)) w...","Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w..."
2,1.623049e+09,3,1623048918,748908519,indiv-illustrate,[],-1,-1,False,False,human,1,1,9,[u: 1\nv: 9],-1,-1,False,False,2,3,human,"Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w...","Situation(Network(|E'|=0 in G(|V|=2, |E|=1)) w..."
3,1.623049e+09,4,1623048920,975822925,indiv-illustrate,[u: 1\nv: 9],-1,-1,False,False,human,3,-1,-1,[u: 1\nv: 9],-1,-1,True,False,3,4,human,"Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w...","Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w..."
4,1.623049e+09,5,1623048961,5374193,pretest-1,[],-1,-1,False,False,human,1,7,0,[u: 7\nv: 0],-1,-1,False,False,0,0,human,"Situation(Network(|E'|=1 in G(|V|=7, |E|=12)) ...","Situation(Network(|E'|=0 in G(|V|=7, |E|=12)) ..."


In [11]:
def construct_actions_and_metadata_from_table(df, verbose=False):
    """Construct all states and action for a log table.

    Note: The argument state_name='state' for the current state, 
    state_name='next_state' for the next state in
    the 3-tuple logs of <state, action, next_state>
    """
    is_submission_list = []
    cost_list = []
    is_mst_list = []
    opt_cost_list = []
    error_list = []
    is_spanning_list = []
    action_list = []
    attempt_no_list = []

    for i, row in df.iterrows():
        # Rename the activity to match with the updated representation package.
        activity = row['header.frame_id']
        activity = world_renamer[activity]
        world = worlds[activity]

        is_submission = identify_submission_state(row)
        is_submission_list.append(is_submission)
        if verbose and is_submission:
            print('Submitted', activity, state)

        prev_action = action_list[i-1] if i > 0 else None
        action = construct_action(
            row, world.env.state.network.graph, prev_action=prev_action)

        # Remap attempt-submit to submit for tests.
        if is_submission and 'test' in activity:
            next_row = df.iloc[i+1] if i < len(df)-1 else None
            if next_row is not None and activity != next_row['header.frame_id']:
                action = SubmitAction(action.agent)

        action_list.append(action)

        state = row['state']

        is_spanning = state.network.is_spanning()
        is_mst = state.network.is_mst()
        opt_cost = state.network.get_mst_cost()
        cost = state.network.get_cost()
        if is_spanning:
            error = 1.0 * (cost - opt_cost) / opt_cost
        else:
            error = None

        cost_list.append(cost)
        opt_cost_list.append(opt_cost)
        error_list.append(error)

        attempt_no_list.append(state.attempt_no)

        is_mst_list.append(is_mst)
        is_spanning_list.append(is_spanning)

    df['action'] = action_list
    df['attempt_no'] = attempt_no_list
    df['cost'] = cost_list  # Submitted cost.
    df['opt_cost'] = opt_cost_list  # Optimal i.e. MST cost.
    df['error'] = error_list
    df['is_submission'] = is_submission_list
    df['is_mst'] = is_mst_list
    df['is_spanning'] = is_spanning_list

    return df

### Construct a transition table from the log table for each participant.

In [12]:
transitions = {}
for participant in sorted(logs):
    df = logs[participant].copy()
    
    df = construct_state_from_table(df, state_name='state')
    df = construct_state_from_table(df, state_name='next_state')
    df = construct_actions_and_metadata_from_table(df)
    
    df.insert(0, 'participant', participant)

    transitions[participant] = df
    print('Processed participant', participant)

Processed participant 1
Processed participant 2
Processed participant 3
Processed participant 4
Processed participant 5
Processed participant 6
Processed participant 7
Processed participant 8
Processed participant 9


### Clean the log tables, drop/rename columns.

In [13]:
transitions_cleaned = dict()
for participant in sorted(transitions):
    df = transitions[participant].copy()

    # Reset time to have duration in the time column.
    df['Time'] -= df.iloc[0]['Time']
    
    name_map = {
            'Time': 'time',
            'header.frame_id': 'activity',
        }
    df.rename(columns=name_map, inplace=True)
    
    # Rename.
    df['activity'] = [world_renamer[a] for a in df['activity']]

    # Remove state element columns.
    df.drop(
        ['header.seq', 
         'header.stamp.secs', 
         'header.stamp.nsecs',
         'action.agent_name',
         'state.edges', 
         'state.suggested.u',
         'state.suggested.v', 
         'state.terminal',
         'state.submit_suggested', 
         'action.type',
         'action.edge.u', 
         'action.edge.v',
         'turn_agent', 
         'next_state.edges',
         'next_state.suggested.u', 
         'next_state.suggested.v',
            'next_state.terminal',
         'next_state.submit_suggested', 
         'action_no', 
         'step_no'],
        axis=1, inplace=True)

    transitions_cleaned[participant] = df


transitions_cleaned[8].head()

,participant,time,activity,state,next_state,action,attempt_no,cost,opt_cost,error,is_submission,is_mst,is_spanning
0,8,0.000000,tutorial,"Situation(Network(|E'|=0 in G(|V|=2, |E|=1)) w...","Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w...","Action(pick(9,1),Human)",1,0,3,NaN,False,False,False
1,8,35.974815,tutorial,"Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w...","Situation(Network(|E'|=0 in G(|V|=2, |E|=1)) w...","Action(clear,Human)",1,3,3,0.0,False,True,True
2,8,59.190591,tutorial,"Situation(Network(|E'|=0 in G(|V|=2, |E|=1)) w...","Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w...","Action(pick(9,1),Human)",1,0,3,NaN,False,False,False
3,8,61.770048,tutorial,"Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w...","Situation(Network(|E'|=1 in G(|V|=2, |E|=1)) w...","Action(submit,Human)",1,3,3,0.0,False,True,True
4,8,101.642923,pretest-1,"Situation(Network(|E'|=0 in G(|V|=7, |E|=12)) ...","Situation(Network(|E'|=1 in G(|V|=7, |E|=12)) ...","Action(pick(4,1),Human)",1,0,15,NaN,False,False,False


In [14]:
# df = transitions[8].copy()
# df = transitions_cleaned[8].copy()
# df = df[df.activity == 'collaboration-1']
# # df = df[df['header.frame_id'] == 'collaboration-1']

# df

### Export the transition tables in CSV data format.

In [15]:
with transitions_pickle_file.open('wb') as handle:
    pickle.dump(transitions_cleaned, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('Saved transition tables to {}'.format(transitions_pickle_file))

Saved transition tables to ../processed_data/justhink21_transitions.pickle


### Construct state transition lists per activity.

In [16]:
# List of transitions, indexed by participant and then the activity name.
transitions_lists = dict()

for participant, log_df in transitions.items():
    print('Processing participant {}:'.format(participant))
    transitions_lists[participant] = dict()

    for name in world_renamer:

        df = log_df[log_df['header.frame_id'] == name].reset_index()

        trans_list = []
        for i, row in df.iterrows():
            if i == 0:
                trans_list.append(row['state'])
            trans_list.append(row['action'])
            trans_list.append(row['next_state'])

        if len(trans_list) != 0:
            print('  Added {} states for {} at {}'.format(
                len(trans_list)+1//2, participant, name))
            transitions_lists[participant][world_renamer[name]] = trans_list

Processing participant 1:
  Added 19 states for 1 at pretest-1
  Added 19 states for 1 at pretest-2
  Added 19 states for 1 at pretest-3
  Added 17 states for 1 at pretest-4
  Added 17 states for 1 at pretest-5
  Added 17 states for 1 at posttest-1
  Added 17 states for 1 at posttest-2
  Added 17 states for 1 at posttest-3
  Added 17 states for 1 at posttest-4
  Added 17 states for 1 at posttest-5
  Added 9 states for 1 at indiv-illustrate
  Added 219 states for 1 at collab-activity
  Added 181 states for 1 at collab-activity-2
Processing participant 2:
  Added 7 states for 2 at pretest-1
  Added 39 states for 2 at pretest-2
  Added 19 states for 2 at pretest-3
  Added 37 states for 2 at pretest-4
  Added 21 states for 2 at pretest-5
  Added 21 states for 2 at posttest-1
  Added 17 states for 2 at posttest-2
  Added 17 states for 2 at posttest-3
  Added 17 states for 2 at posttest-4
  Added 19 states for 2 at posttest-5
  Added 9 states for 2 at indiv-illustrate
  Added 111 states for 

### Export the lists of state transitions in pickle format.

In [17]:
with transition_lists_pickle_file.open('wb') as handle:
    pickle.dump(transitions_lists, handle, protocol=pickle.HIGHEST_PROTOCOL)

print('Saved transition lists to {}'.format(transition_lists_pickle_file))

Saved transition lists to ../processed_data/justhink21_transition_lists.pickle
